<a href="https://colab.research.google.com/github/AVAM16/FreeCodeCamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
stopwords_english = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def text_cleaner(text):
  text = re.sub(r'([^\s\w])+', ' ', text)
  twxt = " ".join([lemmatizer.lemmatize(word) for word in text.split()
                    if not word in stopwords_english])
  text = text.lower()
  return text

In [ ]:
train_dataset = pd.read_csv(train_file_path, sep = "\t", header=None, names=["label", "sentence"])
train_dataset["label"] = train_dataset["label"].map({"ham":0, "spam":1})
test_dataset = pd.read_csv(test_file_path, sep = "\t", header=None, names=["label", "sentence"])
test_dataset["label"] = test_dataset["label"].map({"ham":0, "spam":1})

In [ ]:
train_data = train_dataset['sentence'].apply(lambda x: text_cleaner(x))

In [ ]:
max_words = 1000
max_len = 500
token = keras.preprocessing.text.Tokenizer(num_words = max_words)
token.fit_on_texts(train_data)

In [ ]:
sequences = token.texts_to_sequences(train_data)
sequences_matrix = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

In [ ]:
i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50)(i)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)

In [ ]:
model.fit(sequences_matrix, train_dataset["label"],
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001)])

In [ ]:
def preprocessing(X):
  x = X.apply(lambda x: text_cleaner(x))
  x = token.texts_to_sequences(x)
  return keras.preprocessing.sequence.pad_sequences(x, maxlen=max_len)

model.evaluate(preprocessing(test_dataset['sentence']), test_dataset["label"])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print("Message:", msg)
    print("Prediction:", prediction)
    print("Expected answer:", ans)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
